In [52]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ShuffleSplit,StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold,train_test_split,GroupShuffleSplit,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,mean_squared_error,mean_absolute_error,log_loss,confusion_matrix,accuracy_score,f1_score

import gc

from tqdm.notebook import tqdm


In [53]:
pd.__version__

'1.0.1'

In [54]:
test_app = pd.read_csv('../../raw_data/test_xianchang/test/test_app.csv')
test_sms = pd.read_csv('../../raw_data/test_xianchang/test/test_sms.csv')
test_user = pd.read_csv('../../raw_data/test_xianchang/test/test_user.csv')
test_voc = pd.read_csv('../../raw_data/test_xianchang/test/test_voc.csv')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [96]:
test_user = pd.read_csv('../../raw_data/test_xianchang/test/test_user.csv')

In [98]:
test_user_fusai.arpu_202005.value_counts()

1.000000     101
19.000000     37
13.600000     23
59.000000     20
9.000000      17
            ... 
12.920000      1
31.190001      1
64.889999      1
19.379999      1
22.600000      1
Name: arpu_202005, Length: 893, dtype: int64

In [97]:
test_user.arpu_202007.value_counts()

1.00      199
19.00     129
59.00      63
49.00      54
9.00       30
         ... 
3.40        1
32.73       1
83.27       1
414.92      1
64.82       1
Name: arpu_202007, Length: 1698, dtype: int64

In [57]:
####如果有NA要改 数据格式
test_user = test_user.replace('\\N',np.nan)
test_user.arpu_202007 = test_user.arpu_202007.astype(np.float64)

In [58]:
####读取数据
train_app = pd.read_csv('../../raw_data/201908_202003/train_app.csv')
train_sms = pd.read_csv('../../raw_data/201908_202003/train_sms.csv')
train_user = pd.read_csv('../../raw_data/201908_202003/train_user.csv')
train_voc = pd.read_csv('../../raw_data/201908_202003/train_voc.csv')

test_app_juesai = pd.read_csv('../../raw_data/202006/train_app.csv')
test_sms_juesai = pd.read_csv('../../raw_data/202006/train_sms.csv')
test_user_juesai = pd.read_csv('../../raw_data/202006/train_user.csv')
test_voc_juesai = pd.read_csv('../../raw_data/202006/train_voc.csv')


test_app_chusai = pd.read_csv('../../raw_data/202004/train_app.csv')
test_sms_chusai = pd.read_csv('../../raw_data/202004/train_sms.csv')
test_user_chusai = pd.read_csv('../../raw_data/202004/train_user.csv')
test_voc_chusai = pd.read_csv('../../raw_data/202004/train_voc.csv')

test_user_fusai = pd.read_csv('../../raw_data/202005/train_user.csv')
test_voc_fusai  = pd.read_csv('../../raw_data/202005/train_voc.csv')
test_app_fusai  = pd.read_csv('../../raw_data/202005/train_app.csv')
test_sms_fusai  = pd.read_csv('../../raw_data/202005/train_sms.csv')

test_user_fusai = test_user_fusai.replace('\\N',np.nan)
test_user_fusai.arpu_202005 = test_user_fusai.arpu_202005.astype(np.float32)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
order_dict = {'成都': 22,
 'nan': 21,
 '绵阳': 20,
 '达州': 19,
 '宜宾': 18,
 '凉山': 17,
 '乐山': 16,
 '南充': 15,
 '广元': 14,
 '自贡': 13,
 '德阳': 12,
 '泸州': 11,
 '眉山': 10,
 '雅安': 9,
 '内江': 8,
 '遂宁': 7,
 '资阳': 6,
 '攀枝花': 5,
 '巴中': 4,
 '广安': 3,
 '阿坝': 2,
 '甘孜': 1}

In [60]:
def get_voc_feats(train_voc):
    """
    构建通话记录voc表的特征
    """
    combine_list = []
    ### voc表时间转换
    train_voc['start_datetime'] = pd.to_datetime(train_voc['start_datetime'])
    train_voc['weekday'] = train_voc.start_datetime.dt.weekday
    train_voc['hour'] = train_voc.start_datetime.dt.hour
    train_voc['minute'] = train_voc.start_datetime.dt.minute
    train_voc['second'] = train_voc.start_datetime.dt.second
    train_voc['date'] = train_voc.start_datetime.dt.date
    ### 给多少人打过电话/打过多少个电话
    num_pep_voc_train = train_voc.groupby(['phone_no_m'])['opposite_no_m'].agg(num_call_voc = 'count',num_pep_voc = 'nunique').reset_index()
    combine_list.append(num_pep_voc_train)
    #### 打电话的人里面次数最大小次/平均
    tmp_train = train_voc.groupby(['phone_no_m','opposite_no_m'],as_index=False)['calltype_id'].count()
    num_mean_max_min_for_one_pep_train = \
        tmp_train.groupby(['phone_no_m'])['calltype_id'].agg(num_call_for_1pep_mean = 'mean',
                                                                            num_call_for_1pep_min = 'min',
                                                                            num_call_for_1pep_max = 'max').reset_index()
    combine_list.append(num_mean_max_min_for_one_pep_train)
    #### 一个手机登录了几个IMEI
    num_imei_train = train_voc.groupby(['phone_no_m'],as_index=True)['imei_m'].agg(num_imei_voc = 'nunique').reset_index()
    combine_list.append(num_imei_train)
    
    #### calltype_id count ratio/数量差/比例差
    tmp_phone_call_id = train_voc.groupby(['phone_no_m','calltype_id']).size().reset_index()
    tmp_phone_call_id.columns = ['phone_no_m','calltype_id','num_call_per_calltype']
    ## 每个用户的通话总数
    
    tmp_phone = train_voc.groupby(['phone_no_m']).size().reset_index()
    tmp_phone.columns = ['phone_no_m','number_call']
    
    tmp_phone_call_id = tmp_phone_call_id.merge(tmp_phone,how='left',on='phone_no_m')
    
    tmp_phone_call_id['ratio_call_per_calltype'] = tmp_phone_call_id.num_call_per_calltype/tmp_phone_call_id.number_call
    tmp_all = pd.pivot(tmp_phone_call_id[['phone_no_m','calltype_id','num_call_per_calltype','ratio_call_per_calltype']],index='phone_no_m',columns='calltype_id').fillna(0)
    tmp_list = []
    for a,b in tmp_all.columns:
        tmp_list.append(str(a)+'_'+str(b))
    tmp_all.columns = tmp_list
    tmp_all = tmp_all.reset_index()
    tmp_all['diff_num_calls_1_2'] = tmp_all['num_call_per_calltype_1'] - tmp_all['num_call_per_calltype_2']
    tmp_all['diff_ratio_calls_1_2'] = tmp_all['ratio_call_per_calltype_1'] - tmp_all['ratio_call_per_calltype_2']
    combine_list.append(tmp_all)
    ####增加电话时长的一些统计信息
    call_dur = train_voc.groupby('phone_no_m')['call_dur'].agg(
                                                                call_dur_mean = 'mean',
                                                                call_dur_quantile25 = lambda x:np.quantile(x,0.25),
                                                                call_dur_quantile75 = lambda x:np.quantile(x,0.75),
                                                                call_dur_median = 'median',
                                                                call_dur_max = 'max',
                                                                call_dur_min = 'min',
                                                                call_dur_skew = lambda x:x.skew(),
                                                                call_dur_kurt = lambda x:x.kurt(),
                                                                call_dur_std = 'std'
                                                                            ).reset_index()
    combine_list.append(call_dur)
    #### calltype_id打电话时长均值的差/比例的差

    tmp2 = train_voc.groupby(['phone_no_m','calltype_id'],as_index=False)['call_dur'].mean()
    tmp3 = pd.pivot(tmp2,index='phone_no_m',columns='calltype_id').fillna(0)
    col_list = []
    for a,b in tmp3.columns:
        col_list.append(str(a)+'_'+str(b))
    tmp3.columns=col_list
    tmp3['diff_mean_dur_1_2'] = tmp3['call_dur_1'] - tmp3['call_dur_2']
    tmp3['ratio_mean_dur_1_2'] = tmp3['call_dur_1'] / tmp3['call_dur_2']
    diff_ratio_call_dur =tmp3[['diff_mean_dur_1_2','ratio_mean_dur_1_2']].reset_index()
    combine_list.append(diff_ratio_call_dur)
    
    
    #### city is null ratio 

    train_voc['is_na_city_raio'] = train_voc.city_name.isna()+0
    tmp_is_null_ratio = train_voc.groupby(['phone_no_m'])['is_na_city_raio'].mean().reset_index()
    combine_list.append(tmp_is_null_ratio)
    ### 每天打电话次数的均值/方差/方差均值之比/有打电话的天数

    tmp_call_day = train_voc.groupby(['phone_no_m','date'],as_index=False)['opposite_no_m'].count()
    tmp_1 = tmp_call_day.groupby(['phone_no_m'])['opposite_no_m'].agg(mean_call_per_day='mean',std_call_per_day='std',max_call_per_day = 'max').reset_index()
    #tmp_1['std_mean_ratio_call_per_day'] = tmp_1['std_call_per_day']/tmp_1['mean_call_per_day']
    tmp_2 = tmp_call_day.groupby(['phone_no_m'])['date'].agg(voc_nunique_day='nunique').reset_index()
    combine_list.append(tmp_1)
    combine_list.append(tmp_2)

    ### add voc hour_min_max/std/mean 每天打电话时间的最早小时和最晚小时的方差和均值 众数小时的方差和均值

    tmp_min = train_voc.groupby(['phone_no_m','date'],as_index=False)['hour'].min()
    tmp_min_std = tmp_min.groupby(['phone_no_m'])['hour'].agg(voc_hour_min_std='std').reset_index()

    tmp_max = train_voc.groupby(['phone_no_m','date'],as_index=False)['hour'].max()
    tmp_max_std = tmp_max.groupby(['phone_no_m'])['hour'].agg(voc_hour_max_std='std').reset_index()

    tmp_min = train_voc.groupby(['phone_no_m','date'],as_index=False)['hour'].min()
    tmp_min_mean = tmp_min.groupby(['phone_no_m'])['hour'].agg(voc_hour_min_mean='mean').reset_index()

    tmp_max = train_voc.groupby(['phone_no_m','date'],as_index=False)['hour'].max()
    tmp_max_mean = tmp_max.groupby(['phone_no_m'])['hour'].agg(voc_hour_max_mean='mean').reset_index()
    
    tmp_mode = train_voc.groupby(['phone_no_m','date'],as_index=False)['hour'].agg(lambda x:x.mode()[0])
    tmp_mode_std = tmp_mode.groupby(['phone_no_m'])['hour'].agg(voc_hour_mode_std='std').reset_index()
    
    tmp_mode = train_voc.groupby(['phone_no_m','date'],as_index=False)['hour'].agg(lambda x:x.mode()[0])
    tmp_mode_mean = tmp_mode.groupby(['phone_no_m'])['hour'].agg(voc_hour_mode_mean='mean').reset_index()
    
    combine_list.append(tmp_min_std)
    combine_list.append(tmp_max_std)
    combine_list.append(tmp_min_mean)
    combine_list.append(tmp_max_mean)
    
    combine_list.append(tmp_mode_std)
    combine_list.append(tmp_mode_mean)
    
    #### add hour 比例分布
    hour_tmp1 = train_voc.groupby(['phone_no_m','hour'],as_index=False)['opposite_no_m'].count()
    hour_tmp2 = hour_tmp1.groupby(['phone_no_m'])['opposite_no_m'].transform('sum')
    hour_tmp1['opposite_no_m'] = hour_tmp1['opposite_no_m']/hour_tmp2
    hour_tmp1.columns = ['phone_no_m','hour','call_ratio']
    tmp_a = pd.pivot(hour_tmp1,index='phone_no_m',columns='hour')
    assert tmp_a.shape[1]==24
    tmp_a = tmp_a.fillna(0)
    list1 = []
    for name1,name2 in tmp_a:
        list1.append(str(name1)+'_hour_'+str(name2))
    tmp_a.columns = list1
    tmp_a = tmp_a.reset_index()
    combine_list.append(tmp_a)
    
    ### 打电话所在地情况
    def transform_list(series):
        tmp = series.value_counts()
        max_val = tmp.max()
        return sorted(list(tmp[tmp==max_val].index),key = lambda x:order_dict[x])[0]
    
    tmp = train_voc[['phone_no_m','city_name']]
    tmp['city_name'] = tmp['city_name'].astype(str)
    tmp_1 = tmp.groupby(['phone_no_m'])['city_name'].agg(lambda x:transform_list(x)).reset_index()
    tmp_1.columns = ['phone_no_m','city_name_call_mode']
    combine_list.append(tmp_1)
    
    return combine_list

In [61]:
def get_sms_feats(train_sms):
    """
    构建短信记录sms表的特征
    """
    ### 结合各部分特征
    combine_list = []
    ### 转换时间
    train_sms['request_datetime'] = pd.to_datetime(train_sms['request_datetime'])
    train_sms['weekday'] = train_sms.request_datetime.dt.weekday
    train_sms['hour'] = train_sms.request_datetime.dt.hour
    train_sms['minute'] = train_sms.request_datetime.dt.minute
    train_sms['second'] = train_sms.request_datetime.dt.second
    train_sms['date'] = train_sms.request_datetime.dt.date
    #### 短信 calltype count ratio、收发短信数量/比例差
    temp = train_sms.groupby(['phone_no_m']).size().reset_index()
    temp = temp.rename({0:'count_sms'},axis = 1)
    #计算各个用户不同calltype下短信数量
    temp_id = train_sms.groupby(['phone_no_m','calltype_id']).size().reset_index()
    temp_id = temp_id.rename({0:'count_calltype_sms'},axis = 1)
    #合并
    temp2 = temp.merge(temp_id,how ='left',on ='phone_no_m')
    #计算各个用户下不同calltype短信数量 占 总短信数量比例
    temp2['ratio_sms_per_calltype'] = temp2['count_calltype_sms']/temp2['count_sms']
    #重组
    temp_all = pd.pivot(temp2[['phone_no_m','calltype_id','count_calltype_sms','ratio_sms_per_calltype']], index = 'phone_no_m', columns='calltype_id').fillna(0)
    col_name = [col[0] + '_'+ str(col[1]) for col in temp_all.columns]
    temp_all.columns = col_name
    #计算不同calltype下短信数量差值
    temp_all['diff_count_sms_1_2'] = temp_all['count_calltype_sms_1'] - temp_all['count_calltype_sms_2']
    #计算不同calltype下短信比例差值
    temp_all['diff_ratio_sms_1_2'] = temp_all['ratio_sms_per_calltype_1'] - temp_all['ratio_sms_per_calltype_2']
    temp_all = temp_all.reset_index()
    combine_list.append(temp_all)
    ### 短信 给多少个人短信/多少条/平均
    num_pep_sms_train = train_sms.groupby(['phone_no_m'])['opposite_no_m'].agg(num_txt_sms = 'count',num_pep_sms='nunique').reset_index()
    combine_list.append(num_pep_sms_train)
    #### 短信的人里面次数最大小次/平均
    tmp_train = train_sms.groupby(['phone_no_m','opposite_no_m'],as_index=False)['calltype_id'].count()
    num_mean_max_min_for_one_pep_sms_train = tmp_train.groupby(['phone_no_m'],as_index=False)['calltype_id'].agg(['mean','min','max']).add_prefix('num_txt_for_1pep_').reset_index()
    combine_list.append(num_mean_max_min_for_one_pep_sms_train)

    return combine_list

In [62]:
def get_app_feats(train_app):
    """
    构建app记录app表的特征
    """
    combine_list = []
    #### app 登录过多少个APP/用过的总流量
    nunique_app_train = train_app.groupby(['phone_no_m'])['busi_name'].agg(num_app='nunique',num_app_log = 'count').reset_index()
    combine_list.append(nunique_app_train)
    sum_flow_train = train_app.groupby(['phone_no_m'])['flow'].agg(sum_flow = 'sum').reset_index()
    combine_list.append(sum_flow_train)
    return combine_list

In [63]:
def fe_train_month(train_user,train_voc,train_sms,train_app,month,year):
    """
    对各月数据进行各部分特征进行组合
    """
    if 'type' in train_user.columns:
        keep_list = ['phone_no_m','city_name','county_name','idcard_cnt',f'arpu_{str(year)+str(month)}','label','type']
    elif 'label' in train_user.columns:
        keep_list = ['phone_no_m','city_name','county_name','idcard_cnt',f'arpu_{str(year)+str(month)}','label']
    else:
        keep_list = ['phone_no_m','city_name','county_name','idcard_cnt',f'arpu_{str(year)+str(month)}']
    train_user = train_user[keep_list].copy()
    train_user.rename({f'arpu_{str(year)+str(month)}':'last_month_amt'},axis=1,inplace=True)

    
    combine_list_train = []
    
    combine_list_train.extend(get_voc_feats(train_voc))
    combine_list_train.extend(get_sms_feats(train_sms))
    combine_list_train.extend(get_app_feats(train_app))
    
    for _ in combine_list_train:
        train_user = train_user.merge(_,how='left',on='phone_no_m')
    train_user['month'] = f'{year}-{month}'
    return train_user

In [64]:
#### 将app中null app标记为unknown 方便区别未使用app和 未知的app
train_app = train_app[~((train_app.busi_name.isna())&(train_app.month_id.isna()))]
test_app = test_app[~((test_app.busi_name.isna())&(test_app.month_id.isna()))]
test_app_chusai = test_app_chusai[~((test_app_chusai.busi_name.isna())&(test_app_chusai.month_id.isna()))]
test_app_fusai = test_app_fusai[~((test_app_fusai.busi_name.isna())&(test_app_fusai.month_id.isna()))]
test_app_juesai = test_app_juesai[~((test_app_juesai.busi_name.isna())&(test_app_juesai.month_id.isna()))]

train_app.loc[(train_app.busi_name.isna())&(~train_app.month_id.isna()),'busi_name'] = 'unknown'
test_app.loc[(test_app.busi_name.isna())&(~test_app.month_id.isna()),'busi_name'] = 'unknown'
test_app_chusai.loc[(test_app_chusai.busi_name.isna())&(~test_app_chusai.month_id.isna()),'busi_name'] = 'unknown'
test_app_fusai.loc[(test_app_fusai.busi_name.isna())&(~test_app_fusai.month_id.isna()),'busi_name'] = 'unknown'
test_app_juesai.loc[(test_app_juesai.busi_name.isna())&(~test_app_juesai.month_id.isna()),'busi_name'] = 'unknown'

#### 区分各月数据
train_voc_201908 = train_voc[(train_voc.start_datetime>='2019-08-01 00:00:00')&(train_voc.start_datetime<'2019-09-01 00:00:00')].copy()
train_voc_201909 = train_voc[(train_voc.start_datetime>='2019-09-01 00:00:00')&(train_voc.start_datetime<'2019-10-01 00:00:00')].copy()
train_voc_201910 = train_voc[(train_voc.start_datetime>='2019-10-01 00:00:00')&(train_voc.start_datetime<'2019-11-01 00:00:00')].copy()
train_voc_201911 = train_voc[(train_voc.start_datetime>='2019-11-01 00:00:00')&(train_voc.start_datetime<'2019-12-01 00:00:00')].copy()
train_voc_201912 = train_voc[(train_voc.start_datetime>='2019-12-01 00:00:00')&(train_voc.start_datetime<'2020-01-01 00:00:00')].copy()
train_voc_202001 = train_voc[(train_voc.start_datetime>='2020-01-01 00:00:00')&(train_voc.start_datetime<'2020-02-01 00:00:00')].copy()
train_voc_202002 = train_voc[(train_voc.start_datetime>='2020-02-01 00:00:00')&(train_voc.start_datetime<'2020-03-01 00:00:00')].copy()
train_voc_202003 = train_voc[(train_voc.start_datetime>='2020-03-01 00:00:00')&(train_voc.start_datetime<'2020-04-01 00:00:00')].copy()
train_sms_201908 = train_sms[(train_sms.request_datetime>='2019-08-01 00:00:00')&(train_sms.request_datetime<'2019-09-01 00:00:00')].copy()
train_sms_201909 = train_sms[(train_sms.request_datetime>='2019-09-01 00:00:00')&(train_sms.request_datetime<'2019-10-01 00:00:00')].copy()
train_sms_201910 = train_sms[(train_sms.request_datetime>='2019-10-01 00:00:00')&(train_sms.request_datetime<'2019-11-01 00:00:00')].copy()
train_sms_201911 = train_sms[(train_sms.request_datetime>='2019-11-01 00:00:00')&(train_sms.request_datetime<'2019-12-01 00:00:00')].copy()
train_sms_201912 = train_sms[(train_sms.request_datetime>='2019-12-01 00:00:00')&(train_sms.request_datetime<'2020-01-01 00:00:00')].copy()
train_sms_202001 = train_sms[(train_sms.request_datetime>='2020-01-01 00:00:00')&(train_sms.request_datetime<'2020-02-01 00:00:00')].copy()
train_sms_202002 = train_sms[(train_sms.request_datetime>='2020-02-01 00:00:00')&(train_sms.request_datetime<'2020-03-01 00:00:00')].copy()
train_sms_202003 = train_sms[(train_sms.request_datetime>='2020-03-01 00:00:00')&(train_sms.request_datetime<'2020-04-01 00:00:00')].copy()
train_app_201908 = train_app[train_app.month_id=='2019-08']
train_app_201909 = train_app[train_app.month_id=='2019-09']
train_app_201910 = train_app[train_app.month_id=='2019-10']
train_app_201911 = train_app[train_app.month_id=='2019-11']
train_app_201912 = train_app[train_app.month_id=='2019-12']
train_app_202001 = train_app[train_app.month_id=='2020-01']
train_app_202002 = train_app[train_app.month_id=='2020-02']
train_app_202003 = train_app[train_app.month_id=='2020-03']

In [65]:
#### 各月数据特征加工
test_fe = fe_train_month(test_user,test_voc,test_sms,test_app,'07','2020')
print('Done')
##### 可能要改
test_fe_chusai = fe_train_month(test_user_chusai,test_voc_chusai,test_sms_chusai,test_app_chusai,'04','2020')
print('Done')
test_fe_fusai = fe_train_month(test_user_fusai,test_voc_fusai,test_sms_fusai,test_app_fusai,'05','2020')
print('Done')
test_fe_juesai = fe_train_month(test_user_juesai,test_voc_juesai,test_sms_juesai,test_app_juesai,'06','2020')
print('Done')
train_fe_201908 = fe_train_month(train_user,train_voc_201908,train_sms_201908,train_app_201908,'08','2019')
print('Done')
train_fe_201909 = fe_train_month(train_user,train_voc_201909,train_sms_201909,train_app_201909,'09','2019')
print('Done')
train_fe_201910 = fe_train_month(train_user,train_voc_201910,train_sms_201910,train_app_201909,'10','2019')
print('Done')
train_fe_201911 = fe_train_month(train_user,train_voc_201911,train_sms_201911,train_app_201911,'11','2019')
print('Done')
train_fe_201912 = fe_train_month(train_user,train_voc_201912,train_sms_201912,train_app_201912,'12','2019')
print('Done')
train_fe_202001 = fe_train_month(train_user,train_voc_202001,train_sms_202001,train_app_202001,'01','2020')
print('Done')
train_fe_202002 = fe_train_month(train_user,train_voc_202002,train_sms_202002,train_app_202002,'02','2020')
print('Done')
train_fe_202003 = fe_train_month(train_user,train_voc_202003,train_sms_202003,train_app_202003,'03','2020')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [66]:
#### 组合2019.8-2019.3月数据
train_fe = pd.DataFrame()
for _ in [train_fe_201908,train_fe_201909,train_fe_201910,train_fe_201911,train_fe_201912,train_fe_202001,train_fe_202002,train_fe_202003]:
    train_fe = train_fe.append(_,ignore_index=True)
#### 处理特征county_name和city_name
train_fe['county_name'] = train_fe.city_name.astype(str) + '_'+train_fe.county_name.astype(str)
test_fe['county_name'] = test_fe.city_name.astype(str) + '_'+test_fe.county_name.astype(str)
test_fe_chusai['county_name'] = test_fe_chusai.city_name.astype(str) + '_'+test_fe_chusai.county_name.astype(str)
test_fe_fusai['county_name'] = test_fe_fusai.city_name.astype(str) + '_'+test_fe_fusai.county_name.astype(str)
test_fe_juesai['county_name'] = test_fe_juesai.city_name.astype(str) + '_'+test_fe_juesai.county_name.astype(str)


count_columns = ['county_name','city_name','city_name_call_mode']
for i in count_columns:
    train_fe[i+'_count_full'] = train_fe[i].map(pd.concat([train_fe[i], test_fe_chusai[i],test_fe_fusai[i],test_fe_juesai[i],test_fe[i]], ignore_index=True).value_counts(dropna=False))
    test_fe[i+'_count_full'] = test_fe[i].map(pd.concat([train_fe[i], test_fe_chusai[i],test_fe_fusai[i],test_fe_juesai[i],test_fe[i]], ignore_index=True).value_counts(dropna=False))
    test_fe_chusai[i+'_count_full'] = test_fe_chusai[i].map(pd.concat([train_fe[i], test_fe_chusai[i],test_fe_fusai[i],test_fe_juesai[i],test_fe[i]], ignore_index=True).value_counts(dropna=False))
    test_fe_fusai[i+'_count_full'] = test_fe_fusai[i].map(pd.concat([train_fe[i], test_fe_chusai[i],test_fe_fusai[i],test_fe_juesai[i],test_fe[i]], ignore_index=True).value_counts(dropna=False))
    test_fe_juesai[i+'_count_full'] = test_fe_juesai[i].map(pd.concat([train_fe[i], test_fe_chusai[i],test_fe_fusai[i],test_fe_juesai[i],test_fe[i]], ignore_index=True).value_counts(dropna=False))

train_fe_clean = train_fe.copy()
print(train_fe_clean.shape)
#### label encoding
cat_list = ['city_name','county_name','city_name_call_mode']
for feature in cat_list:
    label_encod = LabelEncoder()
    label_encod.fit(list(train_fe_clean[feature].astype(str).values) + list(test_fe[feature].astype(str).values) + \
                    list(test_fe_chusai[feature].astype(str).values) + list(test_fe_fusai[feature].astype(str).values) +\
                   list(test_fe_juesai[feature].astype(str).values))
    train_fe_clean[feature] = label_encod.transform(list(train_fe_clean[feature].astype(str).values))
    test_fe[feature] = label_encod.transform(list(test_fe[feature].astype(str).values))
    test_fe_chusai[feature] = label_encod.transform(list(test_fe_chusai[feature].astype(str).values))
    test_fe_fusai[feature] = label_encod.transform(list(test_fe_fusai[feature].astype(str).values))
    test_fe_juesai[feature] = label_encod.transform(list(test_fe_juesai[feature].astype(str).values))
### 填空值
train_fe_clean.fillna(-999,inplace=True)
test_fe.fillna(-999,inplace=True)
test_fe_chusai.fillna(-999,inplace=True)
test_fe_fusai.fillna(-999,inplace=True)
test_fe_juesai.fillna(-999,inplace=True)

(48848, 85)


In [67]:
#### 去掉无用字段
train_fe_clean = train_fe_clean.drop(['month'],axis=1)
test_fe_chusai = test_fe_chusai.drop(['month'],axis=1)
test_fe_fusai = test_fe_fusai.drop(['month'],axis=1)
test_fe_juesai = test_fe_juesai.drop(['month'],axis=1)

In [68]:
train_fe_clean = train_fe_clean.drop_duplicates()
train_fe_clean.shape

(40866, 84)

In [69]:
#### 去掉无用字段
X_train = train_fe_clean.drop(['phone_no_m'],axis=1)
y_train =  X_train['label']
X_train.drop(['label'],axis=1,inplace=True)
X_test = test_fe.drop(['phone_no_m','month'],axis=1)
X_test_chusai = test_fe_chusai.drop(['phone_no_m','label'],axis=1)
y_test_chusai = test_fe_chusai['label']
X_test_fusai = test_fe_fusai.drop(['phone_no_m','label'],axis=1)
y_test_fusai = test_fe_fusai['label']

X_test_juesai = test_fe_juesai.drop(['phone_no_m','label'],axis=1)
y_test_juesai = test_fe_juesai['label']

In [70]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(X_test_chusai.shape)
print(y_test_chusai.shape)
print(X_test_fusai.shape)
print(y_test_fusai.shape)
print(X_test_juesai.shape)
print(y_test_juesai.shape)

(40866, 82)
(40866,)
(2798, 82)
(2045, 82)
(2045,)
(1450, 82)
(1450,)
(1895, 82)
(1895,)


In [71]:
#### 合并新四月份数据
X_train = X_train.append(X_test_chusai,ignore_index=True)
y_train = y_train.append(y_test_chusai,ignore_index=True)
X_train = X_train.append(X_test_fusai,ignore_index=True)
y_train = y_train.append(y_test_fusai,ignore_index=True)
X_train = X_train.append(X_test_juesai,ignore_index=True)
y_train = y_train.append(y_test_juesai,ignore_index=True)

In [72]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(46256, 82)
(46256,)
(2798, 82)


In [73]:
y_train = y_train[X_train.last_month_amt!=-999]
X_train = X_train[X_train.last_month_amt!=-999]

In [74]:
print(X_train.shape)
print(y_train.shape)

(43523, 82)
(43523,)


In [75]:
#### LGB参数
bayes_params = \
{'colsample_bytree': 0.3052191680417529,
 'min_child_samples': int(77.99927234100667),
 'min_child_weight': 0.35416028763298746,
 'min_split_gain': 0.06605021932915423,
 'num_leaves': int(7.019404890990211),
 'reg_alpha': 0.24669417297789042,
 'reg_lambda': 1.870655415074352,
 'subsample': 0.7992481953020549}

In [31]:
# X_test = X_test.replace('\\N',-999)

In [32]:
# X_test.last_month_amt = X_test.last_month_amt.astype(np.float32)

In [76]:
#### 五折 LGB
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=32)
fea_impor = 0
k = 1
oof = np.zeros(y_train.shape)
pred = 0
score_list = []
for train_index,test_index in kf.split(X_train,y_train):
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train.iloc[train_index,:]
    y_train2 = y_train.iloc[train_index]
    X_test2 = X_train.iloc[test_index,:]
    y_test2 = y_train.iloc[test_index]
    clf = lgb.LGBMClassifier(n_estimators=10000, random_state=32,learning_rate=0.06,\
                             importance_type = 'gain',n_jobs = -1,bagging_freq=1,metric='None',**bayes_params)
    def f1_score_custom(y_true,y_pred):
        #print(y_pred,y_true.shape)
        y_pred = y_pred.round()
        return 'f1_macro', f1_score(y_true,y_pred,average='macro'), True
    clf.fit(X_train2,y_train2,eval_set = [(X_test2,y_test2)],
            early_stopping_rounds=200,verbose=50,eval_metric=lambda y_true,y_pred:f1_score_custom(y_true,y_pred))
    #joblib.dump(clf, f'lightgbm_adam_fold{k-1}.pkl')
    temp = clf.predict_proba(X_test2)[:,1]
    oof[test_index] = temp
    score_list.append(f1_score(y_test2,temp.round(),average='macro'))
    fea_impor += clf.feature_importances_/kf.n_splits
    pred += clf.predict_proba(X_test)/kf.n_splits
    #break
print('Mean f1_macro:',np.mean(score_list))
print('oof',f1_score(y_train,oof.round(),average='macro'))

fold_1*********************************************
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 200 rounds
[50]	valid_0's f1_macro: 0.775622
[100]	valid_0's f1_macro: 0.806537
[150]	valid_0's f1_macro: 0.815918
[200]	valid_0's f1_macro: 0.819322
[250]	valid_0's f1_macro: 0.821262
[300]	valid_0's f1_macro: 0.824071
[350]	valid_0's f1_macro: 0.828572
[400]	valid_0's f1_macro: 0.83141
[450]	valid_0's f1_macro: 0.832368
[500]	valid_0's f1_macro: 0.834716
[550]	valid_0's f1_macro: 0.836356
[600]	valid_0's f1_macro: 0.837354
[650]	valid_0's f1_macro: 0.838012
[700]	valid_0's f1_macro: 0.8381
[750]	valid_0's f1_macro: 0.839589
[800]	valid_0's f1_macro: 0.839093
[850]	valid_0's f1_macro: 0.84017
[900]	valid_0's f1_macro: 0.839341
[950]	valid_0's f1_macro: 0.841395
[1000]	valid_0's f1_macro: 0.841158
[1050]	valid_0's f1_macro: 0.841319
[1100]	valid_0's f1_macro: 0.843443
[1150]	val

[1300]	valid_0's f1_macro: 0.854937
[1350]	valid_0's f1_macro: 0.856386
[1400]	valid_0's f1_macro: 0.854947
[1450]	valid_0's f1_macro: 0.855819
[1500]	valid_0's f1_macro: 0.856158
[1550]	valid_0's f1_macro: 0.856321
[1600]	valid_0's f1_macro: 0.856713
[1650]	valid_0's f1_macro: 0.85583
[1700]	valid_0's f1_macro: 0.856788
[1750]	valid_0's f1_macro: 0.856688
[1800]	valid_0's f1_macro: 0.857553
[1850]	valid_0's f1_macro: 0.857717
[1900]	valid_0's f1_macro: 0.858683
[1950]	valid_0's f1_macro: 0.858905
[2000]	valid_0's f1_macro: 0.859158
[2050]	valid_0's f1_macro: 0.859232
[2100]	valid_0's f1_macro: 0.860435
[2150]	valid_0's f1_macro: 0.860763
[2200]	valid_0's f1_macro: 0.861329
[2250]	valid_0's f1_macro: 0.861019
[2300]	valid_0's f1_macro: 0.861256
[2350]	valid_0's f1_macro: 0.862039
[2400]	valid_0's f1_macro: 0.86242
[2450]	valid_0's f1_macro: 0.861803
[2500]	valid_0's f1_macro: 0.861092
[2550]	valid_0's f1_macro: 0.862388
[2600]	valid_0's f1_macro: 0.863006
Early stopping, best iteration

In [77]:
### 特征重要性
fea = pd.DataFrame(fea_impor,columns=['feature_impo'],index=X_train.columns).sort_values('feature_impo',ascending=False)
fea.head(50)

,feature_impo
city_name_call_mode,16309.090979
num_call_for_1pep_mean,14625.732654
mean_call_per_day,13241.281789
voc_hour_min_std,6596.303250
idcard_cnt,6474.516357
county_name_count_full,6391.136864
last_month_amt,6340.150135
voc_nunique_day,5530.826383
num_pep_sms,5499.349807
std_call_per_day,5159.481992


In [78]:
# threshold_list = []
# f1_score_list = []
# score_df = pd.DataFrame()
# for _ in tqdm(np.arange(0.05,1,0.01)):
#     threshold_list.append(_)
#     f1_score_list.append(f1_score(test_user_fusai.label,pd.Series(pred[:,1]).map(lambda x:1 if x>=_ else 0),average='macro'))
# score_df['thres'] = threshold_list
# score_df['f1_score'] = f1_score_list
# score_df.sort_values('f1_score',ascending=False)

In [79]:
test_user = pd.read_csv('../../raw_data/test_xianchang/test/test_user.csv')

In [109]:
#### 金额为空值(-999)直接标记为黑样本
#### 预测概率大于0.7标记为黑样本
#### 没交
sub = pd.DataFrame()
sub['phone_no_m'] = test_user['phone_no_m']
sub['label'] = list(pd.Series(pred[:,1]).map(lambda x:1 if x>=0.77 else 0).astype(np.int64))
sub['label'].value_counts()

0    2042
1     756
Name: label, dtype: int64

In [ ]:
0.8 742 0.905
    839 0.892
0.7 801 0.90*
0.75 766 0.906

In [110]:
sub.to_csv('2020-11-3-v5-756.csv',index=False)

In [106]:
sub.label.value_counts()

0    2032
1     766
Name: label, dtype: int64

In [36]:
#### 金额为空值(-999)直接标记为黑样本
#### 预测概率大于0.7标记为黑样本
sub = pd.read_csv('data/raw_data_fusai/test/submit_example.csv')
sub.label = list(pd.Series(pred[:,1]).map(lambda x:1 if x>=0.7 else 0).astype(np.int64))
sub.loc[X_test.last_month_amt==-999,'label'] = 1

FileNotFoundError: [Errno 2] File data/raw_data_fusai/test/submit_example.csv does not exist: 'data/raw_data_fusai/test/submit_example.csv'